In [1]:
cd  /Users/driscoll/Documents/GitHub/fnc/matlab
FNC_init

[**Demo %s**](#demo-condition-bound)


```{index} ! MATLAB; cond
```
MATLAB has a function `cond` to compute matrix condition numbers. By default, the 2-norm is used. As an example, the family of *Hilbert matrices* is famously badly conditioned. Here is the $6\times 6$ case.

In [2]:
A = hilb(6)
kappa = cond(A)

A = 6×6 double
    1.0000    0.5000    0.3333    0.2500    0.2000    0.1667
    0.5000    0.3333    0.2500    0.2000    0.1667    0.1429
    0.3333    0.2500    0.2000    0.1667    0.1429    0.1250
    0.2500    0.2000    0.1667    0.1429    0.1250    0.1111
    0.2000    0.1667    0.1429    0.1250    0.1111    0.1000
    0.1667    0.1429    0.1250    0.1111    0.1000    0.0909

kappa = 1.4951e+07

Because $\kappa\approx 10^8$, it's possible to lose nearly 8 digits of accuracy in the process of passing from $\mathbf{A}$ and $\mathbf{b}$ to $\mathbf{x}$. That fact is independent of the algorithm; it's inevitable once the data are expressed in finite precision. 

Let's engineer a linear system problem to observe the effect of a perturbation. We will make sure we know the exact answer.

In [3]:
x = (1:6)';
b = A * x;

Now we perturb the system matrix and vector randomly by $10^{-10}$ in norm.

In [4]:
dA = randn(size(A));  dA = 1e-10 * (dA / norm(dA));
db = randn(size(b));  db = 1e-10 * (db / norm(db));

We solve the perturbed problem using pivoted LU and see how the solution was changed.

In [5]:
new_x = ((A + dA) \ (b + db));
dx = new_x - x;

Here is the relative error in the solution.

In [6]:
relative_error = norm(dx) / norm(x)

relative_error = 5.1667e-05

And here are upper bounds predicted using the condition number of the original matrix.

In [7]:
upper_bound_b = (kappa * norm(db) / norm(b))
upper_bound_A = (kappa * norm(dA) / norm(A))

upper_bound_b = 1.5810e-04

upper_bound_A = 9.2353e-04

Even if we didn't make any manual perturbations to the data, machine roundoff does so at the relative level of $\macheps$.

In [8]:
dx = A\b - x;
relative_error = norm(dx) / norm(x)
rounding_bound = kappa * eps

relative_error = 4.5027e-11

rounding_bound = 3.3198e-09

Larger Hilbert matrices are even more poorly conditioned:

In [9]:
A = hilb(14);
kappa = cond(A)

kappa = 2.4325e+17

Note that $\kappa$ exceeds $1/\macheps$. In principle we therefore may end up with an answer that has relative error greater than 100%.

In [10]:
rounding_bound = kappa * eps

rounding_bound = 54.0132

Let's put that prediction to the test.

In [11]:
x = (1:14)';  b = A * x;
dx = A\b - x;
relative_error = norm(dx) / norm(x)

relative_error = 61.0121

As anticipated, the solution has zero accurate digits in the 2-norm.